In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('../input/200k-short-texts-for-humor-detection/dataset.csv')
df

You are given this task by your boss, and you are confident you will deliver a good machine learning model with good accuracy. As your eyes scan the data, your mind goes through all the relavent technologies - Logistic regression, LSTM, N grams etc. Just as you are about to take the task, you hear an interruption. 
"Let me take care of this first" Says Bob, your Arch Enemy. Bob knows how to code, but doesn't know machine learning. However, he is confident that he will deliver a good result. The Boss obliges. you being a mahcine learning engineer are expected to beat Bob's result; otherwise it brings your entire job into question!


# Non Machine Learning Baseline

Right off the bat, Bob sees that many humorous examples are in a question-answer format. Let's first verify how much information that gives us. 

In [ ]:
import re
humor_pred = []
for row in df.iterrows():
    string = row[1][0]
    if(bool(re.match(".+\?.+", string))):
        humor_pred.append(True)
    else:
        humor_pred.append(False)

humor_true = df.iloc[:,1]

from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
import seaborn as sns, matplotlib.pyplot as plt

plt.figure(figsize=(11,9))
sns.heatmap(confusion_matrix(humor_pred,humor_true),annot=True,fmt='d')
plt.show()
print(classification_report(humor_pred,humor_true))

We see that its not bad, but a lot of unfunny text is being classified as funny. Let's see what they look like.

(1)Some Examples of unfunny text classified as funny:
1. Want to know why athletes chose to #takeaknee? look at our broken justice system.
2. Struggling to talk to your teenager? the greatest lesson i ever learned.
3. Does drinking soda cause depression? not so fast
4. What would happen if the disney princes were queer -- and hitting the town? (nsfw)
5. A “one taiwan” policy? let’s take out the chinese?
6. Need a new artichoke recipe? we found some of the best
7. What's musk's 'missing piece?' tesla insider shares insights


(2)Some examples of funny text:
1. What do you call a turtle without its shell? dead.
2. What is a pokemon master's favorite kind of pasta? wartortellini!
3. Why do native americans hate it when it rains in april? because it brings mayflowers.
4. How are music and candy similar? we throw away the rappers.
5. What do you call an explanation of an asian cooking show? a wok-through.
6. Why did little miss muffet have gps on her tuffet? to keep her from losing her whey.
7. Did you know diarrhea is genetic? it runs in your jeans
8. What was the sci-fi remake of a streetcar named desire? interstelllllllaaaaaaar
9. What do you call a clan of barbarians you cant see? invisigoths


(3)Examples of Funny text without QA form:

1. My family tree is a cactus, we're all pricks.
2. I just ended a 5 year relationship i'm fine, it wasn't my relationship :p
3. Dating tip: surprise your date! show up a day early.
4. All pants are breakaway pants if you're angry enough
5. Ugh, i just spilled red wine all over the inside of my tummy.
6. Interesting fact: by the year 2020 all actors on american tv shows will be australian.
7. I'd tell you a chemistry joke but i know i won't get a reaction


We can understand that (3) is hard. Let's put that aside. Now, we need to improve our approach to better distinguish (1) and (2). 

Some observations:

(2) has made up words, especially in the 2nd part. 

(2) (and 3) have emojis like :), :p


In [ ]:
for row in df.iterrows():
    string = row[1][0]
    
    #if(not row[1][1] and bool(re.match(".+\?.+", string))):
    if(row[1][1] and  not bool(re.match(".+\?.+", string))):
        print(string)
        

# Machine Learning


In [ ]:
df=pd.read_csv('../input/200k-short-texts-for-humor-detection/dataset.csv')
df

Funny unigrams, Bigrams, Trigrams


In [ ]:
# Functions
import re

from nltk.corpus import stopwords

def preprocess(line):
    line=re.sub('[\n|\t]',' ',line)
    line = re.sub(' +',' ',line)
    return line
def remove_punctuations_line(line):
    line = re.sub('[^a-zA-Z0-9 ]','',line)
    return line


def ngrams(sentence,n):
    return [sentence.split(' ')[i:i+n] for i in range(len(sentence.split(' ')) - n+1)]


def remove_punctuations(ngrams):
    punct = string.punctuation
    return [ngram   for ngram in ngrams   if ngram[0] not in punct and ngram[1] not in punct]
def remove_stopword_ngrams(ngrams):
    stop_words = set(stopwords.words('english'))
    return [ngram   for ngram in ngrams   if ngram[0] not in stop_words and ngram[1] not in stop_words]
def remove_capitalization(line):
    return line.lower()

